In [55]:
import numpy as np
from scipy.stats.distributions import norm
import matplotlib.pylab as plt
import pandas as pd
import warnings; warnings.simplefilter('ignore')
import statsmodels.api as sm
import sys
sys.path.append('../zdrojaky')
from tsplot import tsplot

# Téma 5: Stacionarita, sezónní modely a velké opáčko

Stacionarita, unit-root process, ADF testy, detrendování diferencemi

# Modely SARIMA(p,d,q)(P,D,Q)s

Pokud časová řada vykazuje sezónnost, tj. cyklické opakování určitého vývoje, ARIMA modely stále nebudou stačit. Konkrétně říkáme, že časová řada bez trendu vykazuje sezónnost s periodou $s$, pokud

$$
\mathbb{E}[X_t]=\mathbb{E}[X_{t-s}].
$$

V časových řadách se sezónností je situace komplikovanější, neboť tam je sezónnost dalším důvodem nestacionarity.

Typicky předpokládáme, že perioda $s$ je konstantní, nicméně to samozřejmě není úplná pravda. Data navíc mohou mít více period, např. týdenní a měsíční.

## Přiblížení stacionaritě (d, D)

Z teorie ARIMA modelů už víme, že ke stacionaritě nás posune diferencování, tedy vytvoření nové řady z rozdílů po sobě jdoucích hodnot,

$$
\nabla X_t = (1-B) X_t = X_t - X_{t-1}.
$$

Toto můžeme dělat $d$ krát, většinou maximálně dvakrát, $\nabla^2 X_t$.

Analogicky můžeme přibližně zestacionárnit i časovou řadu se sezónností s periodou $s$,

$$
\nabla_s X_t = (1-B^s) X_t = X_t - X_{t-s}.
$$

Pozor, index je dolní. Opět můžeme diferencovat víckrát, $\nabla_s^D$, zpravidla maximálně jednou. Spojíme-li to dohromady, dostaneme diferencování

$$
\nabla_s^D \nabla^d X_t.
$$

Z AR, MA, ARMA a ARIMA víme, že modelovat závislosti dat v řadě můžeme do libovolné hloubky. Teoreticky bychom mohli sezónnost zkusit postihnout dostatečně složitým modelem, sahajícím pro veličiny hlouběji v minulosti. To je ovšem v rozporu s požadavkem na jednoduchost modelů. Např. při $s=12$ bychom závislost tří po sobě jdoucích sezón modelovali 36 AR  nebo MA koeficienty! Mnohem jednodušší je uvažovat běžný a sezónní vývoj zvlášť a použít multiplikativní model SARIMA(p,d,q)(P,D,Q)s:

$$
\Phi_P(B^s) \phi_p(B) \nabla_s^D \nabla^d X_t = \theta_q(B) \Theta_Q (B^s)\varepsilon_t
$$

kde

- $\Phi_P(B^s) = (1-\Phi_1 B^s - \ldots - \Phi_p B^{sP})$ je sezónní AR operátor řádu $P$,
- $\phi_p = (1 - \phi_1 B - \ldots - \phi_p B^p)$ je běžný AR operátor řádu $p$,
- $\nabla_s^D = (1-B^s)^D$ je operátor sezónní diference řádu $D$,
- $\nabla^d = (1-B)^d$ je operátor běžné diference řádu $d$,
- $\Theta_Q(B^s) = (1 + \Theta_1 B^s + \ldots + \Theta_Q B^{sQ})$ je sezónní MA operátor řádu $Q$,
- $\theta_q(B) = (1 + \theta_1 B + \ldots + \theta_q B^{q})$ je běžný MA operátor řádu $q$,

Zcela patrný je zde (poněkud naivní) předpoklad, že i sezóny se vyvíjí podle neměnného modelu.

## Příklad - počty pasažérů

Podívejme se na příklad s [měsíčními počty pasažérů (v tisících) z období leden 1949 - prosinec 1960](https://datamarket.com/data/set/22u3/international-airline-passengers-monthly-totals-in-thousands-jan-49-dec-60#!ds=22u3&display=line), který jsme si představili na první přednášce. Na grafu vidíme jednoznačně trend i sezónnost. Na spodním grafu je vidět i predikce na (nesmyslných) 100 kroků dopředu (tj. 1 na každý měsíc) a to včetně 95% pásu spolehlivosti.

Odhad byl proveden pomocí pythonu a `statsmodels.tsa.statespace.sarimax`. 

![acf](img/passengers-acf.png)

![arima](img/passengers-arima2.png)

A ještě rezidua:

![arima](img/passengers-resid.png)
![qq](img/passengers-qqhist.png)

Sezonni rw, sezonni trend, obecné sezonní modely...

Možná přidáme eXogeneous vars?